# Desafío n°2

#### Grupo A: Barreiro, Roberto; García-Nagy Julián; Machabanski, Martín; Tiraboschi, Carolina

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

Decidimos trabajar sólo con los datos de propiedades correspondientes a CABA 

In [4]:
data = pd.read_csv("data_desafio2.csv")
data.head(3)

,property_type,place_name,state_name,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,description,title,ambientes,gimnasio,terraza,patio,parrilla,salon,laundry,pileta,cochera,tren_subte,cantidad_amenities
0,PH,Mataderos,Capital Federal,62000.0,55.0,40.0,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,2.0,0,0,1,0,0,0,0,0,0,1
1,apartment,Mataderos,Capital Federal,72000.0,55.0,55.0,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,2.0,0,0,0,0,0,0,0,0,0,0
2,PH,Liniers,Capital Federal,95000.0,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,3.0,0,0,1,0,0,0,0,0,0,1


In [17]:
data.dtypes

property_type             object
place_name                object
state_name                object
price_aprox_usd          float64
surface_total_in_m2      float64
surface_covered_in_m2    float64
rooms                    float64
description               object
title                     object
ambientes                float64
gimnasio                   int64
terraza                    int64
patio                      int64
parrilla                   int64
salon                      int64
laundry                    int64
pileta                     int64
cochera                    int64
tren_subte                 int64
cantidad_amenities         int64
dtype: object

Eliminamos las columnas que ya no vamos a pasar a dummies ni vamos a necesitar, como 'state_name', ya que todo este dataframe es solo de CABA

In [5]:
dfCABA = data.drop(['state_name','description', 'title', 'rooms'], axis = 1)

Dropeamos los nulos en el precio, superficie total y superficie cubierta

In [6]:
dfCABA.isnull().sum()
df = dfCABA.dropna(axis = 0, subset = ['price_aprox_usd','surface_total_in_m2','surface_covered_in_m2'])
df.shape

(26187, 16)

Sacamos las filas con nulos de nuestras columnas para la superficie

In [7]:
df.columns

Index(['property_type', 'place_name', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'ambientes', 'gimnasio', 'terraza', 'patio',
       'parrilla', 'salon', 'laundry', 'pileta', 'cochera', 'tren_subte',
       'cantidad_amenities'],
      dtype='object')

Quitamos los Outliers

In [23]:
columnas = ['price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2','ambientes']
df_reducido = df[columnas]
df.corr()

x =  df.price_aprox_usd
mask = x.between(x.quantile(.05),x.quantile(.95))
x =  df.surface_total_in_m2
mask_sup_total = x.between(x.quantile(.05),x.quantile(.95))
x =  df.surface_covered_in_m2
mask_sup_cubierta = x.between(x.quantile(.05),x.quantile(.95))
datos = df[mask & mask_sup_total & mask_sup_cubierta]

print(df.shape)
print(mask.sum())
print(xprecio.shape)
print(xsup_total.shape)
print(xsup_cubierta.shape)
datos.shape

(26187, 17)
23582


NameError: name 'xprecio' is not defined

In [9]:
df.isnull().sum()
#todavía mantenemos los ambientes que tienen bastantes nulos para luego ver 
#si podemos probar un modelo en el que dropiemos los nulos y ver cuánto afecta 

property_type               0
place_name                  0
price_aprox_usd             0
surface_total_in_m2         0
surface_covered_in_m2       0
ambientes                7950
gimnasio                    0
terraza                     0
patio                       0
parrilla                    0
salon                       0
laundry                     0
pileta                      0
cochera                     0
tren_subte                  0
cantidad_amenities          0
dtype: int64

Agrupamos en 8 zonas los valores de la columna place_name, seleccionando propuestas de divisiones de paginas de inmobiliarias. Tambien luego se pueden ver modelos que tomen como dataframe una zona en particular. 

In [10]:
barrios_caba_mapper = {
    'Abasto' : 'Eje central',
    'Agronomía' : 'Corredor Noroeste',
    'Almagro' : 'Eje central',
    'Balvanera' : 'Eje central',
    'Barracas' : 'Sur Este',
    'Barrio Norte' : 'Corredor Norte',
    'Belgrano' : 'Corredor Norte',
    'Boca' : 'Sur Este',
    'Boedo' : 'Eje central',
    'Capital Federal' : 'Sin determinar zona',
    'Catalinas' : 'Corredor Norte',
    'Centro / Microcentro' : 'Macrocentro',
    'Chacarita' : 'Corredor Noroeste',
    'Coghlan' : 'Corredor Norte',
    'Constitución' : 'Sur Este',
    'Floresta' : 'Oeste',
    'Las Cañitas' : 'Corredor Norte',
    'Liniers' : 'Oeste',
    'Monserrat' : 'Macrocentro',
    'Monte Castro' : 'Noroeste',
    'Nuñez' : 'Corredor Norte',
    'Once' : 'Eje central',
    'Palermo' : 'Corredor Norte',
    'Palermo Chico' : 'Corredor Norte',
    'Palermo Hollywood' : 'Corredor Norte',
    'Palermo Soho' : 'Corredor Norte',
    'Palermo Viejo' : 'Corredor Norte',
    'Parque Avellaneda' : 'Oeste',
    'Parque Centenario' : 'Eje central',
    'Parque Chacabuco' : 'Eje central',
    'Parque Chas' : 'Corredor Noroeste',
    'Parque Patricios' : 'Sur Este',
    'Paternal' : 'Corredor Noroeste',
    'Puerto Madero' : 'Macrocentro',
    'Recoleta' : 'Corredor Norte',
    'Saavedra' : 'Corredor Norte',
    'San Cristobal' : 'Sur Este',
    'San Nicolás' : 'Macrocentro',
    'San Telmo' : 'Macrocentro',
    'Tribunales' : 'Macrocentro',
    'Velez Sarsfield' : 'Oeste',
    'Versalles' : 'Oeste',
    'Villa Crespo' : 'Corredor Noroeste',
    'Villa del Parque' : 'Noroeste',
    'Villa General Mitre' : 'Noroeste',
    'Villa Lugano' : 'Sur',
    'Villa Luro' : 'Oeste',
    'Villa Ortuzar' : 'Corredor Noroeste',
    'Villa Pueyrredón' : 'Noroeste',
    'Villa Real' : 'Oeste',
    'Villa Riachuelo' : 'Sur',
    'Villa Santa Rita' : 'Noroeste',
    'Villa Soldati' : 'Sur',
    'Villa Urquiza' : 'Corredor Noroeste', 
    'Caballito' : 'Eje central',
    'Colegiales' : 'Corredor Norte',
    'Congreso' : 'Macrocentro',
    'Flores' : 'Eje central',
    'Mataderos' : 'Oeste',
    'Pompeya' : 'Sur',
    'Retiro' : 'Corredor Norte',
    'Villa Devoto' : 'Noroeste'

}

In [11]:
#creamos la columna zonas que va a dividir en 8 zonas inmobiliarias la capital
df['zonas'] = df['place_name'].map(barrios_caba_mapper)


D:\conda\envs\dhdsblend\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Creamos un nuevo dataframe que genere dummies de todas las variables categóricas 

In [13]:
df.property_type.unique()
df_dummies =  pd.get_dummies(df, drop_first = True)

Definimos funciones para luego probar modelos de regresión. Primero una regresion lineal, luego regularizada por ridge y por lasso. En todos los casos la funcion recibe un dataframe y las columnas features y va a imprimir las metricas (errores y R2) para cada modelo. Estos modelos los vamos a alimentar de diferentes formas, cambiando las columnas ingresadas como features y los hiperparámetros. Tambien se pueden probar diferentes filtros de partes del dataset.   

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression

def train_test_error(df1,feature_cols):
    X = df1[feature_cols]
    y = df1.price_aprox_usd
    #dividimos entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    #seleccionamos el modelo
    linreg = LinearRegression()
    #entrenamos el modelo
    linreg.fit(X_train, y_train)
    #predecimos los resultados 
    y_pred = linreg.predict(X_test)
    
    #mostramos las metricas del modelo y las features utilizadas
    # print (feature_cols) Esta linea es por si queremos ver qué columnas le estamos pasando
    
    print ('MAE:', metrics.mean_absolute_error(y_test, y_pred))
    print ('MSE:', metrics.mean_squared_error(y_test, y_pred))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('R2:', metrics.r2_score(y_test, y_pred))
    print ()
    return 

In [15]:
from sklearn import linear_model
from sklearn.metrics import r2_score

def train_test_ridge(df1,feature_cols):
    X = df1[feature_cols]
    y = df1.price_aprox_usd
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    lm_ridge = linear_model.RidgeCV(alphas=v, normalize=True) 
    # Definimos el rango de de búsqueda del hiperparametro explicitamente
    model_2 = lm_ridge.fit(X_train, y_train)
    y_pred = model_2.predict(X_test)
    
    print (feature_cols)
    print('Score model_2:', model_2.score(X_test, y_test))
    
    print ('MAE:', metrics.mean_absolute_error(y_test, y_pred))
    print ('MSE:', metrics.mean_squared_error(y_test, y_pred))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('R2:', metrics.r2_score(y_test, y_pred))
    print ('alpha', lm_ridge.alpha_)
    return



In [16]:

def train_test_lasso(df1,feature_cols):
    X = df1[feature_cols]
    y = df1.price_aprox_usd
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)    
    
    lm_lasso = linear_model.LassoCV(alphas=[0.1, 1, 10],normalize=True, cv=3)
    
    model = lm_lasso.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    #scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
    #print(dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum()))
    
    print (feature_cols)
    print('Score model:', model.score(X_test, y_test))
    
    print ('MAE:', metrics.mean_absolute_error(y_test, y_pred))
    print ('MSE:', metrics.mean_squared_error(y_test, y_pred))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('R2:', metrics.r2_score(y_test, y_pred))
    print ('alpha', model.alpha_)
    return

In [17]:
df.columns

Index(['property_type', 'place_name', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'ambientes', 'gimnasio', 'terraza', 'patio',
       'parrilla', 'salon', 'laundry', 'pileta', 'cochera', 'tren_subte',
       'cantidad_amenities', 'zonas'],
      dtype='object')

In [18]:
cols1 = ['surface_total_in_m2','surface_covered_in_m2']
df_dummies.drop(columns = 'ambientes', inplace = True)

In [19]:
mask_cols = df_dummies.columns != 'price_aprox_usd'
columnas = df_dummies.columns[mask_cols]

In [20]:
train_test_error(df_dummies,columnas)

MAE: 148235.81750809675
MSE: 98348281788.22842
RMSE: 313605.29617375473
R2: 0.2520599951633683



In [21]:
print(train_test_ridge(df_dummies,columnas))


NameError: name 'v' is not defined

In [22]:
print(train_test_lasso(df_dummies,columnas))

D:\conda\envs\dhdsblend\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 277728155210041.75, tolerance: 161646154543.98517
  tol, rng, random, positive)
D:\conda\envs\dhdsblend\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 415681128117384.7, tolerance: 161646154543.98517
  tol, rng, random, positive)
D:\conda\envs\dhdsblend\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 172884169870259.7, tolerance: 159049985162.90195
  tol, rng, random, positive)
D:\conda\envs\dhdsblend\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You migh

Index(['surface_total_in_m2', 'surface_covered_in_m2', 'gimnasio', 'terraza',
       'patio', 'parrilla', 'salon', 'laundry', 'pileta', 'cochera',
       'tren_subte', 'cantidad_amenities', 'property_type_apartment',
       'property_type_house', 'property_type_store', 'place_name_Agronomía',
       'place_name_Almagro', 'place_name_Balvanera', 'place_name_Barracas',
       'place_name_Barrio Norte', 'place_name_Belgrano', 'place_name_Boca',
       'place_name_Boedo', 'place_name_Caballito',
       'place_name_Capital Federal', 'place_name_Catalinas',
       'place_name_Centro / Microcentro', 'place_name_Chacarita',
       'place_name_Coghlan', 'place_name_Colegiales', 'place_name_Congreso',
       'place_name_Constitución', 'place_name_Flores', 'place_name_Floresta',
       'place_name_Las Cañitas', 'place_name_Liniers', 'place_name_Mataderos',
       'place_name_Monserrat', 'place_name_Monte Castro', 'place_name_Nuñez',
       'place_name_Once', 'place_name_Palermo', 'place_name_Pale